In [28]:
%load_ext autoreload
%autoreload 2

import csv
import math
from utility_functions import *
from component_functions import *




def naive_bayes_classifier(testdoc, logprior, loglikelihood, C, V):
    
    
    
    
    
    
    
    
    

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [33]:
# Testing countWrdsOfVocInCls function in component_functions.py

countWrdsOfVocInCls('../sentiment-labelled-sentences/mainDataset.csv', 1)

23098

In [34]:
# Testing likelihoodOfWordInCls in component_functions.py

likelihoodOfWordInCls('the', 1)

-1.3648792186987904

In [35]:
# Testing priorProb function from component_functions.py

priorProb('../sentiment-labelled-sentences/mainDataset.csv', 1)


-0.6020599913279624